In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [6]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:43<00:00, 27.1MB/s]
100% 1.06G/1.06G [00:43<00:00, 26.6MB/s]


In [7]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [8]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [9]:
train_dataset = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

test_dataset = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [10]:
#Now the images are stored in the form of a numpy array, whose value ranges from 0 to 255,
#so normalize to get in between 0 and 1
#to make all pixel values between 0 and 1
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_dataset= train_dataset.map(process)
test_dataset = test_dataset.map(process)

In [11]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 254, 254, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 125, 125, 64)      256       
 chNormalization)                                                
                                                        

In [13]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [14]:
history = model.fit(train_dataset, epochs = 10, validation_data = test_dataset)

Epoch 1/10
625/625 [==============================] - 73s 101ms/step - loss: 1.5763 - accuracy: 0.5731 - val_loss: 0.6849 - val_accuracy: 0.5320
Epoch 2/10
625/625 [==============================] - 63s 100ms/step - loss: 0.6491 - accuracy: 0.6356 - val_loss: 0.7277 - val_accuracy: 0.6492
Epoch 3/10
625/625 [==============================] - 66s 105ms/step - loss: 0.5814 - accuracy: 0.6919 - val_loss: 0.7464 - val_accuracy: 0.6010
Epoch 4/10
625/625 [==============================] - 66s 106ms/step - loss: 0.4909 - accuracy: 0.7577 - val_loss: 0.4891 - val_accuracy: 0.7554
Epoch 5/10
625/625 [==============================] - 70s 112ms/step - loss: 0.4262 - accuracy: 0.7995 - val_loss: 0.4380 - val_accuracy: 0.7930
Epoch 6/10
625/625 [==============================] - 65s 103ms/step - loss: 0.3684 - accuracy: 0.8314 - val_loss: 0.4088 - val_accuracy: 0.8188
Epoch 7/10
625/625 [==============================] - 68s 108ms/step - loss: 0.3122 - accuracy: 0.8644 - val_loss: 0.4605 - val_ac